In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split


def load_images_with_labels(base_directory):
    images = []
    labels = []
    processed_counts = {}
    label_map = {'up': 0, 'straight': 1, 'left': 2, 'right': 3}
    
    for root, dirs, files in os.walk(base_directory):
        folder_name = os.path.basename(root)
        processed_counts[folder_name] = {'total': 0, 'processed': 0, 'skipped': 0}
        
        for filename in files:
            if filename.endswith(('.png', '.pgm')):
                file_path = os.path.join(root, filename)
                try:
                    processed_counts[folder_name]['total'] += 1
                    
                    img = Image.open(file_path).convert('RGB')
                    img = img.resize((64, 64))
                    img_array = np.array(img) / 255.0
                    
                    parts = filename.split('_')
                    if len(parts) > 1:
                        orientation = parts[1]
                        if orientation in label_map:
                            images.append(img_array)
                            labels.append(label_map[orientation])
                            processed_counts[folder_name]['processed'] += 1
                        else:
                            processed_counts[folder_name]['skipped'] += 1
                            print(f"Skipping file with unexpected orientation: {filename} in {folder_name}")
                    else:
                        processed_counts[folder_name]['skipped'] += 1
                        print(f"Skipping file with missing orientation: {filename} in {folder_name}")
                        
                except Exception as e:
                    processed_counts[folder_name]['skipped'] += 1
                    print(f"Error processing {filename} in {folder_name}: {e}")
    
    # Print processing summary
    print("\nProcessing Summary:")
    for folder, counts in processed_counts.items():
        if counts['total'] > 0: 
            print(f"\n{folder}:")
            print(f"  Total files: {counts['total']}")
            print(f"  Successfully processed: {counts['processed']}")
            print(f"  Skipped: {counts['skipped']}")
            if counts['processed'] > 0:
                success_rate = (counts['processed'] / counts['total']) * 100
                print(f"  Success rate: {success_rate:.2f}%")
    
    return np.array(images), np.array(labels)

directory = 'images'
images, labels = load_images_with_labels(directory)
print(len(images))
